In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset
import pandas as pd
import os
import sys
import datasets
import transformers
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [3]:
dataset = load_dataset('SKNahin/bengali-transliteration-data')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['bn', 'rm'],
        num_rows: 5006
    })
})


In [5]:
dataset['train'][1]

{'bn': 'ও গুলা টরেন্ট সাইট এ পাবেন', 'rm': 'o gula Torrent site e paben'}

In [6]:
train_df = dataset['train']
train_df[0]

{'bn': 'স্ক্রোল করে ২০/৩০ সেকেন্ড এর ভিডিও পান নাই???',
 'rm': 'scroll kore 20/30 second er video pann nai???'}

In [8]:
model2 = "t5-base"

# Preprocessing

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model2)

In [10]:
tokenizer("স্ক্রোল করে ২০/৩০ সেকেন্ড এর ভিডিও পান নাই???")

{'input_ids': [3, 2, 3, 2, 3, 2, 87, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 8665, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["scroll kore 20/30 second er video pann nai???"]))

{'input_ids': [[11930, 3, 5543, 15, 460, 87, 1458, 511, 3, 49, 671, 2131, 29, 3, 29, 9, 23, 8665, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [12]:
# print(len(train_df['bn'][1]))
# print(train_df['bn'][1])
# print(train_df[1])
# print(len(train_df['bn']))

inp_max_len = 0
tr_max_len = 0

for i in range(len(train_df)):
  if len(train_df['bn'][i]) > inp_max_len:
    inp_max_len = len(train_df['bn'][i])

for i in range(len(train_df)):
  if len(train_df['rm'][i]) > tr_max_len:
    tr_max_len = len(train_df['rm'][i])

print(inp_max_len)
print(tr_max_len)

246
253


In [108]:
max_input_length = 200
max_target_length = 200

inp_lng = 'rm'
tr_lng = 'bn'

def preprocess_function(examples):
  inputs = examples[inp_lng]
  targets = examples[tr_lng]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs



In [109]:
preprocess_function(dataset['train'][:2])

{'input_ids': [[11930, 3, 5543, 15, 460, 87, 1458, 511, 3, 49, 671, 2131, 29, 3, 29, 9, 23, 8665, 1], [3, 32, 3, 6106, 9, 3, 31610, 353, 3, 15, 2576, 115, 35, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[3, 2, 3, 2, 3, 2, 87, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 8665, 1], [3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 1]]}

# Model Train

In [111]:
tokenized_datasets = dataset.map(preprocess_function)

Map:   0%|          | 0/5006 [00:00<?, ? examples/s]

In [112]:
model1 = "Helsinki-NLP/opus-mt-en-hi"

In [113]:
model2 = "t5-base"

In [114]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model2)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [115]:
batch_size = 8
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [118]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [119]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=200)

In [120]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [121]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [ ]:
model.fit(train_dataset, epochs=1)

# Testing

In [85]:
tokenizer = AutoTokenizer.from_pretrained(model2)
model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-base")

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [86]:
input_text  = "o gula Torrent site e paben"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=200)
print(out)

tf.Tensor(
[[    0 32099    32     3  6106     9     3 31610   353     3    15  2576
    115    35     3    32     3  6106     9     3 31610   353     3    15
   2576   115    35     3    32     3  6106     9     3 31610   353     3
     15  2576   115    35     3    32     3  6106     9     3 31610   353
      3    15  2576   115    35     3    32     3  6106     9     3 31610
    353     3    15  2576   115    35     3    32     3  6106     9     3
  31610   353     3    15  2576   115    35     3    32     3  6106     9
      3 31610   353     3    15  2576   115    35     3    32     3  6106
      9     3 31610   353     3    15  2576   115    35     3    32     3
   6106     9     3 31610   353     3     1]], shape=(1, 115), dtype=int32)


In [88]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

o gula Torrent site e paben o gula Torrent site e paben o gula Torrent site e paben o gula Torrent site e paben o gula Torrent site e paben o gula Torrent site e paben o gula Torrent site e paben o gula Torrent site e paben o gula Torrent site 
